# Outil de récupération d'informations géographiques via l'API Geonames

In [ ]:
#@title Installation des bibliothèques nécessaires et connexion à un compte Google Drive.

''' 
Google Colab notebook.
Python == 3.7.11

BaOIA - La Contemporaine - Université de Nanterre
'''


## Installation des bibliothèques et connexion au compte Google Drive

!pip install geopy==1.17.0
from google.colab import drive
from geopy.geocoders import Nominatim

drive.mount('/content/drive/')
%cd /content/drive/My Drive/geonames/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/geonames


In [ ]:
#@markdown Fichier d'entrée: fichier .txt (texte brut) contenant une liste de lieux à aligner avec l'API Geonames

chemin_fichier_txt_lieux = '/content/drive/My Drive/geonames/extrait_loc.txt'#@param {type:"string"}
#@markdown Fichier de sortie: fichier .txt (texte brut) le sinformations récupérées
nom_fichier_txt_donnees_lieux = 'donnees_lieux_geonames.txt' #@param {type:"string"}
geolocator = Nominatim(user_agent="marinagiardinetti") ##Besoin d'un identifiant

#@markdown Les informations récupérées sont notamment les suivantes: ville, pays, région, département, coordonnées géographiques (latitude et longitude). 


with open (chemin_fichier_txt_lieux, "r") as myfile:
    data=myfile.readlines()
with open(nom_fichier_txt_donnees_lieux, 'w') as my_coords:
  for lieu in data:
    try:
      location = geolocator.geocode(lieu, timeout=10)
      coords = "".join([str(lieu), ": ", str(location.address), ", ", str(location.latitude), ", ", str(location.longitude), "\n"])
      my_coords.write(coords)
    except AttributeError:
      print("Pas de géolocalisation trouvée")

# Avec geopy = possibilité de calculer les distances



amiens

Amiens, Somme, Hauts-de-France, France métropolitaine, France
(49.8941708, 2.2956951)
{'place_id': 258318801, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 114347, 'boundingbox': ['49.846837', '49.9505487', '2.2235574', '2.3457767'], 'lat': '49.8941708', 'lon': '2.2956951', 'display_name': 'Amiens, Somme, Hauts-de-France, France métropolitaine, France', 'class': 'boundary', 'type': 'administrative', 'importance': 0.7249489622249039, 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_boundary_administrative.p.20.png'}
samarobriva

Amiens, Somme, Hauts-de-France, France métropolitaine, France
(49.8941708, 2.2956951)
{'place_id': 258318801, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 114347, 'boundingbox': ['49.846837', '49.9505487', '2.2235574', '2.3457767'], 'lat': '49.8941708', 'lon': '2.2956951', 'display_name': 'Amiens,

In [ ]:
## Création fichier XML pour géologcalisation multiple (carte)
from geopy.geocoders import Nominatim
import xml.etree.cElementTree as ET
!pip install yattag
from yattag import Doc, indent

doc, tag, text = Doc().tagtext()
geolocator = Nominatim(user_agent="marinagiardinetti") ##Besoin d'un identifiant
with open ("/content/drive/My Drive/geonames/extrait_loc.txt", "r") as myfile:
    data=myfile.readlines()
with open("output.xml", "w") as my_coords:
  for lieu in data:
    print(lieu)
    try:
      location = geolocator.geocode(lieu, timeout=10)
      print((location.latitude, location.longitude))
      with tag('lieu', str(lieu)):
          with tag('latitude'):
            text(str(location.latitude))
          with tag('longitude'):
            text(str(location.longitude))
      result = indent(
      doc.getvalue(),
      indentation = ' '*4,
     newline = '\n'
)
      print(result)
      my_coords.write(result)
    except AttributeError:
      print("Pas de géolocalisation trouvée")


In [ ]:
#CRÉER DATAFRAME POUR mutltiple cartes avec coordonnées GPS
from geopy.geocoders import Nominatim
import pandas as pd

geolocator = Nominatim(user_agent="marinagiardinetti") ##Besoin d'un identifiant
with open ("/content/drive/My Drive/geonames/extrait_loc.txt", "r") as myfile:
    data=myfile.readlines()

df_coords={}
with open("output.csv", "w") as my_coords:
  for lieu in data:
    #print(lieu)
    try:
      location = geolocator.geocode(lieu, timeout=10)
      #print((location.latitude, location.longitude))
      df_coords = pd.DataFrame.from_dict({
'place':[lieu],
'lat':[location.latitude],
'lon':[location.longitude],
})
      print(df_coords)
      df_coords.to_csv(my_coords, columns= ['place','lat', 'lon'], index = False, header=False)
    except AttributeError:
      print("Pas de géolocalisation trouvée")


In [ ]:
#### CREATION D'UNE CARTE AVEC PLUSIEURS REPÈRES
## Modification du fichier CSV
import folium
import csv
with open('output.csv',newline='') as f:
    r = csv.reader(f)
    data = [line for line in r]
with open('output.csv','w',newline='') as f:
    w = csv.writer(f)
    w.writerow(['place', 'lat', 'lon'])
    w.writerows(data)

data_for_map = pd.read_csv("output.csv", error_bad_lines=False, delimiter = ",")
print(data_for_map)

# Création de la carte
map_test = folium.Map(location=[46.232193, 2.209667], zoom_start=6). ## Ici: carte de la France. 

def create_map_markers(row, map_name):
    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

found_locations = data_for_map[data_for_map['place']!= "Not Found"]
found_locations.apply(create_map_markers, map_name=map_test, axis='columns')
map_test
map_test

In [ ]:
import folium
from folium import plugins

m = folium.Map(location=[48.856614, 2.3522219], zoom_start=12)

# Lon, Lat order.
lines = [
    {
        "coordinates": [
            [139.76451516151428, 35.68159659061569],
            [139.75964426994324, 35.682590062684206],
        ],
        "dates": ["2017-06-02T00:00:00", "2017-06-02T00:10:00"],
        "color": "red",
    },
    {
        "coordinates": [
            [139.75964426994324, 35.682590062684206],
            [139.7575843334198, 35.679505030038506],
        ],
        "dates": ["2017-06-02T00:10:00", "2017-06-02T00:20:00"],
        "color": "blue",
    },
    {
        "coordinates": [
            [139.7575843334198, 35.679505030038506],
            [139.76337790489197, 35.678040905014065],
        ],
        "dates": ["2017-06-02T00:20:00", "2017-06-02T00:30:00"],
        "color": "green",
        "weight": 15,
    },
    {
        "coordinates": [
            [139.76337790489197, 35.678040905014065],
            [139.76451516151428, 35.68159659061569],
        ],
        "dates": ["2017-06-02T00:30:00", "2017-06-02T00:40:00"],
        "color": "#FFFFFF",
    },
]

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {
            "times": line["dates"],
            "style": {
                "color": line["color"],
                "weight": line["weight"] if "weight" in line else 5,
            },
        },
    }
    for line in lines
]

plugins.TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1M",
    add_last_point=True,
).add_to(m)

m